# 1.  Imports and Data

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler



from utils import stratified_train_test_group_kfold
from utils import pred2class
from utils import model_test_regression
from regression_codes import dtr_gridsearch



with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_target = np.array(list(df['T']))
features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))
train_val_sampleId, test_sampleId, train_val_Y, test_Y = train_test_split(sampleId, T_target, test_size=0.1, stratify = T_target, random_state=40)

0.10.1


# 2. Data Splitting and Scaling

In [2]:


X = features
Y = np.array(T_target)
groups = np.array(sampleId)




train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)
train_val_Y = train_val_Y.flatten()
#val_Y = val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


# 3. Runing Cross-Validation

In [6]:
Criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
Max_depth =[5, 6, 7, 8, None]
Min_samples_split = [i*0.1 for i in range(1, 10, 1)]
Max_features =[0.1, 0.2, 0.4, 'log2', 'sqrt', None]# ['auto', 'sqrt', 'log2', None]


def cross_10_folds_gpc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'regression/ImageNet/aug3/DTR_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = dtr_gridsearch(train_X, train_Y, val_X, val_Y, Criterion,Max_depth, Min_samples_split, Max_features, fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_gpc(train_val_X, train_val_Y)

#with open("Model/Reg/TrainedImageNet/Up/DTR_RegModels_T_best_variables", "wb") as fb:
#    pickle.dump(best_variables, fb)


print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 0.726984126984127, best_val_acc: 0.625
best_train_acc: 0.8031746031746032, best_val_acc: 0.6666666666666666
best_train_acc: 0.8888888888888888, best_val_acc: 0.7083333333333334
fold: 0 done!
<class 'generator'>
best_train_acc: 0.8050314465408805, best_val_acc: 0.625
best_train_acc: 0.8427672955974843, best_val_acc: 0.7083333333333334
best_train_acc: 0.9559748427672956, best_val_acc: 0.75
fold: 1 done!
<class 'generator'>
best_train_acc: 0.7924528301886793, best_val_acc: 0.5416666666666666
best_train_acc: 0.8050314465408805, best_val_acc: 0.625
best_train_acc: 0.8176100628930818, best_val_acc: 0.75
best_train_acc: 0.8364779874213837, best_val_acc: 0.7916666666666666
best_train_acc: 0.8616352201257862, best_val_acc: 0.8333333333333334
fold: 2 done!
<class 'generator'>
best_train_acc: 0.746031746031746, best_val_acc: 0.7083333333333334
best_train_acc: 0.8285714285714286, best_val_acc: 0.75
fold: 3 done!
<class 'generator'>
best_train_acc: 0.765079365079

# 4. Model Testing

In [5]:
best_fold =2

model_path = f'regression/ImageNet/aug3/DTR_{best_fold}_model.sav'
# Fitting different train in the same hyperparameters
best_test, root_mean_squared_error, confusion_matrix_test = model_test_regression(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)

print('best_test: ', best_test)
print('rmse: ', root_mean_squared_error)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}, rmse_mean: {np.mean(root_mean_squared_error)}, rmse_std: {np.std(root_mean_squared_error)}')

print("......")

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
best_test:  [0.4444444444444444, 0.48148148148148145, 0.6296296296296297, 0.5555555555555556, 0.5555555555555556, 0.5185185185185185, 0.5925925925925926, 0.48148148148148145, 0.5555555555555556, 0.5925925925925926]
rmse:  [40.824829046386306, 46.14791034954486, 30.429030972509228, 37.26779962499649, 33.333333333333336, 38.490017945975055, 31.91423692521127, 36.00411499115478, 44.09585518440984, 39.67460238079361]
test_acc_mean: 0.541, std: 0.055, rmse_mean: 37.818173075431474, rmse_std: 4.846595603143858
......
5th confusion matrix:  [[2, 1, 0], [2, 6, 5], [0, 4, 7]]


In [6]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print('avg cm: ', [list(item) for item in np.mean(cm_list, axis=0)])


10
avg cm:  [[1.7, 1.3, 0.0], [2.3, 5.4, 5.3], [1.1, 2.4, 7.5]]


In [3]:
best_fold = 2
PATH = f'Model/Reg/TrainedImageNet/Up/DTR_{best_fold}_model.sav'
#PATH = os.path.join('Model', model_path)
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'absolute_error',
 'max_depth': None,
 'max_features': 0.1,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 0.2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': 1,
 'splitter': 'best'}